In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
import scipy.ndimage as ndi
from time import time
import pyopencl as cl
import pandas as pd
import os

In [11]:
# tolerance used in floating point comparisons
TOL = 0.001

# Max dim for NDRange
DIM = 2

# number of times to do each multiplication
COUNT = 10

def results(N, C, run_time):
    mflops = 2.0 * N * N * N/(1000000.0* run_time)
    print(run_time, "seconds at", mflops, "MFLOPS")
                #to do
    #errsq = error(N, C)
    #if (errsq > TOL):
    #    print("Errors in multiplication:", errsq)

#tamanho do filtro        
window_size = 7
#ler a imagem
h_img = numpy.zeros((1000,1000)).astype(numpy.float32)
h_img[20:600,0:250] = 255
h_img[0:400,250:500] = 255
h_img[130:740,500:750] = 255
h_img[:,750:] = 255
h_img += 40*numpy.random.rand(1000, 1000)
N = h_img.shape[0]
#imagem tem formato 256x256
sigma_dist=2
sigma_int=50
radius = window_size//2

h_img_smooth = numpy.empty(N*N).astype(numpy.float32)



In [4]:
# Setup OpenCL
context = cl.create_some_context()
print("DEVICE", context.devices)

queue = cl.CommandQueue(context)

# Create OpenCL buffers
d_img = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_img)
d_img_smooth = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_img_smooth.nbytes)

DEVICE [<pyopencl.Device 'GeForce RTX 2060 SUPER' on 'NVIDIA CUDA' at 0x15f24856360>]


In [14]:

kernelsource = open("dist_weights.cl").read()
print("<<<KERNEL SOURCE>>>\n", kernelsource)

program = cl.Program(context, kernelsource).build()
BlF = program.bilateral_filter
BlF.set_scalar_arg_dtypes([numpy.int32, numpy.int32, numpy.float32, numpy.float32, None])
print("\n==== Image smooth by bilateral filter, order", N, "on device ======\n")

# Do the multiplication COUNT times
for i in range(COUNT):
    h_img_smooth.fill(0.0)
    start_time = time()
    BlF(queue, (N,N), None, N, N,
        radius, sigma_dist, d_img_smooth)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_img_smooth, d_img_smooth)
    #results(N, h_img_smooth, run_time)


<<<KERNEL SOURCE>>>
 __kernel void bilateral_filter(
            const unsigned int    x_img,
            const unsigned int    y_img,
            const float    radius,
            const float    sigma_dist,
            __global       float* dist_weights)
{
    int i = get_global_id(0);
    int j = get_global_id(1);
    if ((i < x_img) && (j < y_img)){
        dist_weights[i*x_img + j] = exp(0-((i - radius) * (i-radius) + (j-radius) * (j - radius)) / 2 * (sigma_dist*sigma_dist));
    }
}
    


==== Image smooth by bilateral filter, order 1000 on device ======



In [16]:

    h_img_smooth[0:7,0:7]

IndexError: too many indices for array